In [18]:
library(swat)
library(ggplot2)
library(reshape2)
library(dplyr)

In [19]:
Sys.setenv(CAS_CLIENT_SSL_CA_LIST = "/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem")
conn <- CAS('frasepviya35smp.cloud.com', 5570)

NOTE: Connecting to CAS and generating CAS action functions for loaded
      action sets...
NOTE: To generate the functions with signatures (for tab completion), set 
      options(cas.gen.function.sig=TRUE).


In [20]:
# Activate metric tracing and other session parameters
cas.sessionProp.setSessOpt(conn, metrics=TRUE, timeout=1800, caslib='casuser')

NOTE: 'CASUSER(viyademo01)' is now the active caslib.
NOTE: Action 'sessionProp.setSessOpt' used (Total process time):
NOTE:       real time               0.000498 seconds
NOTE:       cpu time                0.000489 seconds (98.19%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  280.44K (0.00%)


list()

In [24]:
Sys.time()
tbl <- defCasTable(conn, tablename="megacorp5_4m", caslib = "AZUREDL")
tblR <- to.casDataFrame(tbl,obs = 10000)
Sys.time()

[1] "2021-10-15 12:59:39 UTC"

NOTE: Executing action 'table.columnInfo'.
NOTE: Action 'table.columnInfo' used (Total process time):
NOTE:       real time               0.001117 seconds
NOTE:       cpu time                0.001053 seconds (94.27%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  847.22K (0.00%)
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               0.055621 seconds
NOTE:       cpu time                0.055572 seconds (99.91%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  2.58M (0.00%)


[1] "2021-10-15 12:59:39 UTC"

In [28]:
str(tblR)

'data.frame':	10000 obs. of  46 variables:
Formal class 'casDataFrame' [package "swat"] with 15 slots
  ..@ .Data      :List of 46
  .. ..$ : num  13916 13975 13909 14049 13975 ...
  .. ..$ : num  13880 13880 13880 13880 13880 ...
  .. ..$ : num  13911 13970 13880 14031 13970 ...
  .. ..$ : num  6 2 6 6 2 4 5 5 4 5 ...
  .. ..$ : num  45 45 45 45 45 45 45 45 45 45 ...
  .. ..$ : chr  "CA00045" "CA00045" "CA00045" "CA00045" ...
  .. ..$ : chr  "Plant" "Plant" "Plant" "Plant" ...
  .. ..$ : num  2 2 2 2 2 2 2 2 2 2 ...
  .. ..$ : num  0.0933 0.0989 0.2005 0.097 0.0989 ...
  .. ..$ : chr  "West" "West" "West" "West" ...
  .. ..$ : chr  "CA" "CA" "CA" "CA" ...
  .. ..$ : chr  "San Francisco" "San Francisco" "San Francisco" "San Francisco" ...
  .. ..$ : num  47 46 46 47 46 46 46 46 46 46 ...
  .. ..$ : chr  "NOVPR0000047" "NOVPR0000046" "NOVPR0000046" "NOVPR0000047" ...
  .. ..$ : num  0 0 0 0 0 0 0 0 0 0 ...
  .. ..$ : num  2 2 2 2 2 2 2 2 2 2 ...
  .. ..$ : num  0.833 0.833 0.833 0.833 0

In [26]:
object.size(tblR)
# 3,618,733,968 bytes

8414560 bytes

In [13]:
cas.table.tableDetails(conn,name='megacorp5_4m',caslib='Azuredl')
# 1,881,513,088 bytes

NOTE: Executing action 'table.tableDetails'.
NOTE: Action 'table.tableDetails' used (Total process time):
NOTE:       real time               0.000958 seconds
NOTE:       cpu time                0.000899 seconds (93.84%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  712.59K (0.00%)


Node,Blocks,Active,Rows,IndexSize,DataSize,VardataSize,CompressedSize,CompressionRatio,Mapped,MappedMemory,Unmapped,UnmappedMemory,Allocated,AllocatedMemory,DeletedRows
ALL,1795,1795,4522868,0,1881513088,0,0,0,1795,1881721360,0,0,0,0,0


In [14]:
tblRdf <- to.data.frame(tblR)

In [15]:
object.size(tblRdf)
# 1,664,823,064 bytes

1664823064 bytes

In [17]:
cas.session.endSession(conn)

[1] "ERROR: Connection failed. Server returned: Session reconnect failed: Could not find the specified session."


ERROR: Error in swat::errorcheck(sw_connection): An error occurred while sending request.
